In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score,precision_score, recall_score,roc_curve, auc, roc_auc_score



C:\Python\Python39\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
def plot_train(history):   
    epoch_count = range(1,len(history.history['accuracy']) + 1)
    sns.lineplot(x=epoch_count, y = history.history['val_loss'],label = 'validation loss')
    sns.lineplot(x=epoch_count, y = history.history['val_accuracy'],label = 'validation acc')
    sns.lineplot(x=epoch_count, y = history.history['loss'],label = 'train loss')
    sns.lineplot(x=epoch_count, y = history.history['accuracy'],label = 'train acc')
    plt.title("SoftMax Regression")
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.grid()
    plt.show()

In [27]:
def sns_val_acc(history,itera,data):   
    epoch_count = range(1,len(history.history['val_accuracy']) + 1)
    #sns.lineplot(x=epoch_count, y = history.history['val_loss'],label = 'validation loss, itr = %d' %itera)
    sns.lineplot(x=epoch_count, y = history.history['val_accuracy'],label = 'validation acc, itr = %d, pesoInit = %s' %(itera,data))

In [30]:
def sns_val_MAE(history,itera,data):   
    epoch_count = range(1,len(history.history['val_MAE']) + 1)
    #sns.lineplot(x=epoch_count, y = history.history['val_loss'],label = 'validation loss, itr = %d' %itera)
    sns.lineplot(x=epoch_count, y = history.history['val_MAE'],label = 'validation MAE = %0.4f, itr = %d, bs = %d ' %(np.min(history.history['val_MAE']),itera,data))

In [21]:
def get_f1_prec_recall(model,X_val_par,y_val_par):
    y_pred = model.predict(X_val_par)

    prediction = np.zeros(X_val_par.shape[0]).astype(int)
    for k in range(X_val_par.shape[0]):
        prediction[k] = np.argmax(y_pred[k])

    f1_macro = f1_score(y_val_par, prediction,average='macro')
    f1_micro = f1_score(y_val_par, prediction,average='micro')
    f1_weihtged = f1_score(y_val_par, prediction,average='weighted')
    precision = precision_score(y_val_par, prediction,average='macro')
    recall = recall_score(y_val_par, prediction,average='macro')
    print(" Precision: \t",precision,"\n","Recall: \t", recall)
    print(" F1 Macro: \t",f1_macro,"\n","F1 Micro: \t",f1_micro,"\n", "F1 Weihtged: ",f1_weihtged)
    return f1_macro,precision,recall

In [22]:
def ROC_curve(model,X_val_par,y_val_par):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    y_pred = model.predict(X_val_par)
    prediction = np.zeros(X_val_par.shape[0]).astype(int)
    for k in range(X_val_par.shape[0]):
        prediction[k] = np.argmax(y_pred[k])
    for i in range(10):
        fpr[i], tpr[i], _ = roc_curve(y_sparse_val[:,i], np.eye(10)[prediction][:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    #Ploteo
    plt.figure(figsize=(10, 6))
    colors = ['darkorange', 'blue', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

    for i in range(10):
        plt.plot(fpr[i], tpr[i], color=colors[i], lw=2, label='ROC curve class '+label_dict[i]+' (AUC = %0.2f)' % (roc_auc[i]))

    plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('- ROC - (AUC-MACRO =  %0.3f")' %np.mean(np.array(list(roc_auc.values()))))
    plt.legend(loc="lower right")
    plt.show() 

    print("Macro averege of the AUC value:  %0.2f" %np.mean(np.array(list(roc_auc.values()))))
    return np.mean(np.array(list(roc_auc.values())))

In [23]:
def save2csv(history,f1,recall,precision,auc,name):
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 
    hist_df['precision'] = precision
    hist_df['recall'] = recall
    hist_df['f1'] = f1_macro
    hist_df['auc'] = auc
    hist_df.to_csv(name,index=False)